In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
import regionate as reg
import sectionate as sec

In [3]:
import geopandas as gpd
from shapely.geometry import Polygon
import regionmask

In [4]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
div_cmap = plt.get_cmap("RdBu_r").copy()
div_cmap.set_bad("grey")

cmap = plt.get_cmap("viridis").copy()
cmap.set_bad("grey")

#### Helper functions

In [5]:
def basin_boundary_grid_indices(
    b, ocean_grid,
    coordnames={
        't': ('geolon',   'geolat'  ),
        'c': ('geolon_c', 'geolat_c'),
        'u': ('geolon_u', 'geolat_u'),
        'v': ('geolon_v', 'geolat_v')
    }):
    i, j, lons_c, lats_c = sec.create_section_composite(
        ocean_grid[coordnames['c'][0]],
        ocean_grid[coordnames['c'][1]],
        np.append(b.lons, b.lons[0]),
        np.append(b.lats, b.lats[0]),
        closed=True
    )
    
    symmetric = ocean_grid[coordnames['t'][0]].shape==ocean_grid[coordnames['c'][0]].shape
    uvpoints = sec.transports_C.MOM6_UVpoints_from_section(i, j, symmetric=symmetric)

    lons_uv, lats_uv = sec.transports_C.MOM6_UVcoords_from_points_uv(
        ocean_grid[coordnames['u'][0]],
        ocean_grid[coordnames['u'][1]],
        ocean_grid[coordnames['v'][0]],
        ocean_grid[coordnames['v'][1]],
        uvpoints,
    )

    return i, j, lons_c[:-1], lats_c[:-1], lons_uv, lats_uv

In [6]:
def wrap_continuously(x, limit_discontinuity=180.):
    new_x = x.copy()
    for i in range(len(new_x)-1):
        if new_x[i+1]-new_x[i] >= 180.:
            new_x[i+1] -= 360.
        elif new_x[i+1]-new_x[i] < -180:
            new_x[i+1] += 360.
    return new_x

In [7]:
def basin_interior_grid_mask(
    b, ocean_grid,
    coordnames={
        't': ('geolon',   'geolat'  ),
        'c': ('geolon_c', 'geolat_c'),
        'u': ('geolon_u', 'geolat_u'),
        'v': ('geolon_v', 'geolat_v')
    }):
    Δlon = np.sum(np.diff(b.lons)[np.abs(np.diff(b.lons)) < 180])
    
    if np.abs(Δlon) < 180.:
        lons = np.append(b.lons, [b.lons[0]])
        lats = np.append(b.lats, [b.lats[0]])
        wrapped_lons = wrap_continuously(lons)
        minlon = np.min(wrapped_lons)
        polygon_geom = Polygon(zip(np.mod(wrapped_lons-minlon, 360.), lats))

        crs = 'epsg:4326'
        polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
        basin_grid_mask = ~np.isnan(regionmask.mask_geopandas(polygon, np.mod(ocean_grid[coordnames['t'][0]]-minlon, 360.), lat=ocean_grid[coordnames['t'][1]], wrap_lon='360'))
        
    else:
        # Flag specifies whether circumgeo regions are bounded by north or south pole
        s = np.sign(Δlon).astype(int)
        
        if s>0:
            min_idx = np.argmin(b.lons)
        else:
            min_idx = np.argmax(b.lons)
        lons = np.roll(b.lons, -min_idx)
        lats = np.roll(b.lats, -min_idx)

        diffs = s*(lons[np.newaxis, :] - lons[:, np.newaxis])
        diffs[np.tril_indices(lons.size)]*=-1
        single_valued = ~np.any(diffs < 0, axis=1)
        
        roll_idx = np.argmax(single_valued[::s])
        lons = np.roll(lons[::s], -roll_idx)[::s]
        lats = np.roll(lats[::s], -roll_idx)[::s]
        lons[::s][-roll_idx:] = lons[::s][-roll_idx:] - s*360.

        min_idx = np.argmin(lons)
        lons = np.append(lons, [lons[-1], lons[-1]-s*360, lons[-1]-s*360, lons[0]])
        lats = np.append(lats, [s*90,     s*90,           lats[0],        lats[0]])

        minlon = np.min(lons)
        polygon_geom = Polygon(zip(lons-minlon, lats))
        crs = 'epsg:4326'
        polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
        basin_grid_mask = ~np.isnan(regionmask.mask_geopandas(polygon, ocean_grid[coordnames['t'][0]]-minlon, lat=ocean_grid[coordnames['t'][1]], wrap_lon='360'))
    
    return basin_grid_mask

In [8]:
def conform_basin_to_ocean_grid(b, ocean_grid):
    b.i, b.j, b.lons, b.lats, b.lons_uv, b.lats_uv = basin_boundary_grid_indices(b, ocean_grid)
    b.mask = basin_interior_grid_mask(b, ocean_grid)

## Load connected regions

In [9]:
import pickle
BasinsFile = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region"
with open(BasinsFile, 'rb') as pickle_file:
    region = pickle.load(pickle_file)
    
# Slightly shift coordinates off of supergrid nodes to avoid rounding errors
for bname, b in region.Basins.items():
    b.lons -= 1.e-3
    b.lats -= 1.e-3

## Conform regions to CM4p25 grid

In [10]:
region_gridded = region.copy()

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p25/c192_OM4_025_grid_No_mg_drag_v20160808_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p25_v7_allfixes/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid = ocean_grid.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                           'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})
    
# Conform basin boundaries to model grid
print("Conforming regions to ocean grid: ", end="")
for (bname, b) in region_gridded.Basins.items():
    print(bname, end=", ")
    conform_basin_to_ocean_grid(b, ocean_grid)
reg.check_global_coverage(region_gridded)
region_gridded.find_all_overlaps(face_indices=True)

# `pickle` boundary indices and area mask
regionFile_CM4p25 = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p25"
with open(regionFile_CM4p25, 'wb') as pickle_file:
    pickle.dump(region_gridded, pickle_file)

Conforming regions to ocean grid: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 2, 30, 31, 32, 33, 3, 4, 5, 6, 7, 8, 9, 35, 34, 

## Conform regions to CM4p125 grid

In [11]:
region_gridded = region.copy()

# Load CMp125 grid
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    
    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid = ocean_grid.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                           'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})

# Conform basin boundaries to model grid
print("Conforming regions to ocean grid: ", end="")
for (bname, b) in region_gridded.Basins.items():
    print(bname, end=", ")
    conform_basin_to_ocean_grid(b, ocean_grid)
reg.check_global_coverage(region_gridded)
region_gridded.find_all_overlaps(face_indices=True)

    
# `pickle` boundary indices and area mask
regionFile_CM4p125 = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p125"
with open(regionFile_CM4p125, 'wb') as pickle_file:
    pickle.dump(region_gridded, pickle_file)

Conforming regions to ocean grid: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 2, 30, 31, 32, 33, 3, 4, 5, 6, 7, 8, 9, 35, 34, 